In [1]:
import numpy as np
import pickle
import scipy.stats
import csv

In [2]:
def make_memory(word_list):
    mem = []
    for i in word_list:
        mem.append(word_matrix[word_dic[i]])
    return np.array(mem)

In [3]:
def normalize_matrix(mat):
    mag = np.sum(mat**2, axis=1)
    mag[mag == 0] = 1
    mag = np.sqrt(mag)
    return np.transpose(np.transpose(mat) / mag)

In [4]:
def echo_intensity(probes, mem, tau=3):
    normed_mem = normalize_matrix(mem)
    return np.sum((probes @ np.transpose(normed_mem))**3, axis=1)

In [5]:
def r_sq(x, y):
    cor = scipy.stats.pearsonr(x, y)
    return cor[0]**2

In [6]:
def mse(x, y):
    return np.sum((x - y)**2) / len(x)

In [7]:
#import LSA

The LSA semantic space can be downloaded from https://osf.io/s6db4. The file was too big to share on Github

In [8]:
with open('LSA semantic space', 'rb') as f:
    word_dic, word_matrix = pickle.load(f)

In [9]:
word_matrix = normalize_matrix(word_matrix)

In [10]:
#import word lists

In [11]:
def get_stims_from_csv(filename):
    df = []
    with open(filename, 'r') as f:
        csvreader = csv.reader(f)
        for i in csvreader:
            df.append(i)
    df = df[1:]
    r_old = [i[1].lower() for i in df if i[2]=='R' and i[3]=='old']
    f_old = [i[1].lower() for i in df if i[2]=='F' and i[3]=='old']
    r_new = [i[1].lower() for i in df if i[2]=='R' and i[3]=='new']
    f_new = [i[1].lower() for i in df if i[2]=='F' and i[3]=='new']
    new_new = [i[1].lower() for i in df if i[2]=='New' and i[3]=='new']
    return r_old + f_old + r_new + f_new + new_new

In [12]:
stims_a = get_stims_from_csv('df_testlist1.csv')
stims_b = get_stims_from_csv('df_testlist2.csv')
stims_c = get_stims_from_csv('df_testlist3.csv')
stims_d = get_stims_from_csv('df_testlist4.csv')

In [13]:
#empirical means
emp_250_250 = [0.58402778, 0.53263889, 0.39722222, 0.33055556, 0.3125]
emp_250_2 = [0.67616959, 0.47660819, 0.29093567, 0.23830409, 0.20248538]
emp_2_250 = [0.69642857, 0.51455026, 0.24074074, 0.20767196, 0.13425926]
emp_2_2 = [0.71544715, 0.53319783, 0.2601626 , 0.18834688, 0.14295393]

In [14]:
# all empirical means in one array (for computing R-squared and MSE)
emp_all = [0.58402778, 0.53263889, 0.39722222, 0.33055556, 0.3125, 0.67616959, 0.47660819, 0.29093567, 0.23830409, 0.20248538, 0.69642857, 0.51455026, 0.24074074, 0.20767196, 0.13425926, 0.71544715, 0.53319783, 0.2601626 , 0.18834688, 0.14295393]

In [16]:
# simulations

In [18]:
#Selective Rehearsal

L = .02  # Learning rate
l_decay = .9 # pre-cue decay rate
R = .01 # post-cue boost rate
cue_decay = .5 # post-cue decay rate

t = 3 # retrieval gradient

cond = [(1, 1, 45, 'short/short'), (1, 8, 40, 'short/long '), (8, 1, 39, 'long/short '), (8, 8, 40, 'long/long  ')]

overall_means = []
sds = []
print('            Selective Rehearsal Simulation MEANS\n')
print('condition    R old      F old      R rel      F rel      Unrel\n')
for c in cond:
    sim_list = []
    for s in range(0, 1000):
        if s < 250:
            probes = make_memory(stims_a)
        elif s < 500:
            probes = make_memory(stims_b)
        elif s < 750:
            probes = make_memory(stims_c)
        else:
            probes = make_memory(stims_d)
        mem = probes[:72].copy()
        r_mem = np.zeros((36, 300))
        f_mem = np.zeros((36, 300))
        l = L + 0
        for i in range(0, c[0]):
            r_encode = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-l, l])
            f_encode = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-l, l])
            r_mem[r_encode == 1] = 1
            f_mem[f_encode == 1] = 1
            l *= l_decay
        r = l + R
        for i in range(0, c[1]):
            r_encode = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-r, r])
            f_encode = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-l, l])
            r_mem[r_encode == 1] = 1
            f_mem[f_encode == 1] = 1
            l *= cue_decay
            r *= cue_decay
        mem[:36] *= r_mem
        mem[36:] *= f_mem
        fams = echo_intensity(probes, mem, tau=t)
        criterion = np.percentile(fams, 100-c[2])
        r_hits = np.sum(fams[:36] > criterion) / 36
        f_hits = np.sum(fams[36:72] > criterion) / 36
        r_fa = np.sum(fams[72:90] > criterion) / 18
        f_fa = np.sum(fams[90:108] > criterion) / 18
        u_fa = np.sum(fams[108:144] > criterion) / 36
        sim_list.append([r_hits, f_hits, r_fa, f_fa, u_fa])
    sim_means = np.mean(sim_list, axis=0)
    sds.append(np.std(sim_list, axis=0, ddof=1))
    print(c[3], sim_means)
    overall_means.append(sim_means)

            Selective Rehearsal Simulation MEANS

condition    R old      F old      R rel      F rel      Unrel

short/short [0.60227778 0.50697222 0.43044444 0.394      0.28408333]
short/long  [0.6235     0.45475    0.35583333 0.30716667 0.20136111]
long/short  [0.62533333 0.56405556 0.25027778 0.23694444 0.12255556]
long/long   [0.68580556 0.56458333 0.26077778 0.22372222 0.11847222]


In [19]:
# print SDS
print('            Selective Rehearsal Simulation Standard Deviations\n')
print('condition    R old      F old      R rel      F rel      Unrel\n')
for i in range(0, 4):
    print(cond[i][3], sds[i])

            Selective Rehearsal Simulation Standard Deviations

condition    R old      F old      R rel      F rel      Unrel

short/short [0.08250373 0.07231178 0.12142764 0.12247948 0.08686609]
short/long  [0.08228911 0.07481775 0.11145779 0.10328875 0.07094774]
long/short  [0.07330819 0.07228168 0.08682353 0.07928718 0.0447977 ]
long/long   [0.06972488 0.07168623 0.08654534 0.07833647 0.04350774]


In [21]:
sim_all = []
for x in overall_means:
    sim_all.extend(x)
print('Selective Rehearsal fits\n')
print('R-squared: ', r_sq(np.array(sim_all), emp_all))
print('Mean Squared Error: ', mse(np.array(sim_all), emp_all))

Selective Rehearsal fits

R-squared:  0.9584387378073439
Mean Squared Error:  0.0015720045014903515


In [24]:
#active forgetting

L = .04 # learning rate
l_decay = .8 # learning rate decay
F = .2 # forgetting rate
f_decay = 0 # forgetting rate decay

t = 3 # retrieval gradient

cond = [(1, 1, 45, 'short/short'), (1, 8, 40, 'short/long '), (8, 1, 39, 'long/short '), (8, 8, 40, 'long/long  ')]

overall_means = []
sds = []
print('            Attention Inhibition Simulation MEANS\n')
print('condition    R old      F old      R rel      F rel      Unrel\n')
for c in cond:
    sim_list = []
    for s in range(0, 1000):
        if s < 250:
            probes = make_memory(stims_a)
        elif s < 500:
            probes = make_memory(stims_b)
        elif s < 750:
            probes = make_memory(stims_c)
        else:
            probes = make_memory(stims_d)
        mem = probes[:72].copy()
        r_mem = np.zeros((36, 300))
        f_mem = np.zeros((36, 300))
        l = L + 0
        f = F + 0
        for i in range(0, c[0]):
            r_encode = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-l, l])
            f_encode = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-l, l])
            r_mem[r_encode == 1] = 1
            f_mem[f_encode == 1] = 1
            l *= l_decay
        for i in range(0, c[1]):
            f_forget = np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-f, f])
            f_mem[f_forget == 1] = 0
            f *= f_decay
        mem[:36] *= r_mem
        mem[36:] *= f_mem
        fams = echo_intensity(probes, mem, tau=t)
        criterion = np.percentile(fams, 100-c[2])
        r_hits = np.sum(fams[:36] > criterion) / 36
        f_hits = np.sum(fams[36:72] > criterion) / 36
        r_fa = np.sum(fams[72:90] > criterion) / 18
        f_fa = np.sum(fams[90:108] > criterion) / 18
        u_fa = np.sum(fams[108:144] > criterion) / 36
        sim_list.append([r_hits, f_hits, r_fa, f_fa, u_fa])
    sim_means = np.mean(sim_list, axis=0)
    sds.append(np.std(sim_list, axis=0, ddof=1))
    print(c[3], sim_means)
    overall_means.append(sim_means)

            Attention Inhibition Simulation MEANS

condition    R old      F old      R rel      F rel      Unrel

short/short [0.58772222 0.50011111 0.43711111 0.39916667 0.29958333]
short/long  [0.52869444 0.44741667 0.39211111 0.36055556 0.25866667]
long/short  [0.71580556 0.50280556 0.25433333 0.20066667 0.10944444]
long/long   [0.73591667 0.52425    0.26461111 0.21022222 0.11352778]


In [25]:
# print SDS
print('            Attention Inhibition Simulation Standard Deviations\n')
print('condition    R old      F old      R rel      F rel      Unrel\n')
for i in range(0, 4):
    print(cond[i][3], sds[i])

            Attention Inhibition Simulation Standard Deviations

condition    R old      F old      R rel      F rel      Unrel

short/short [0.07969496 0.07470734 0.12993405 0.12675899 0.08578933]
short/long  [0.08402059 0.07289451 0.12175972 0.12547442 0.08157061]
long/short  [0.06905345 0.0708813  0.09010533 0.07177479 0.0435399 ]
long/long   [0.06491574 0.06512315 0.08905194 0.07418544 0.04203284]


In [26]:
sim_all = []
for x in overall_means:
    sim_all.extend(x)
print('Attention inhibition fits\n')
print('R-squared: ', r_sq(np.array(sim_all), emp_all))
print('Mean Squared Error: ', mse(np.array(sim_all), emp_all))

Attention inhibition fits

R-squared:  0.9141010886181157
Mean Squared Error:  0.0030851370479070194


In [27]:
# Context Unbinding
l = .02 # learning rate
f = .8 # unbinding rate

context_size = 50 # context vector size
t = 3 # retrieval gradient

cond = [(1, 1, 45, 'short/short'), (1, 8, 40, 'short/long '), (8, 1, 39, 'long/short '), (8, 8, 40, 'long/long  ')]

overall_means = []
sds = []
print('            Unbinding Simulation MEANS\n')
print('condition    R old      F old      R rel      F rel      Unrel\n')
for c in cond:
    sim_list = []
    for s in range(0, 1000):
        if s < 250:
            stims = make_memory(stims_a)
        elif s < 500:
            stims = make_memory(stims_b)
        elif s < 750:
            stims = make_memory(stims_c)
        else:
            stims = make_memory(stims_d)
        context_matrix = np.ones((144, context_size)) * np.random.normal(0, 1/np.sqrt(300), context_size)
        probes = np.concatenate((stims, context_matrix), axis=1)
        mem = probes[:72].copy()
        r_mem = np.zeros((36, 300+context_size))
        f_mem = np.zeros((36, 300+context_size))
        for i in range(0, c[0]):
            r_encode = np.random.choice([0, 1], size=(36, 300+context_size), replace=True, p=[1-l, l])
            f_encode = np.random.choice([0, 1], size=(36, 300+context_size), replace=True, p=[1-l, l])
            r_mem[r_encode == 1] = 1
            f_mem[f_encode == 1] = 1
        for i in range(0, c[1]):
            r_encode = np.random.choice([0, 1], size=(36, 300+context_size), replace=True, p=[1-l, l])
            f_encode = np.concatenate((np.random.choice([0, 1], size=(36, 300), replace=True, p=[1-l, l]), np.zeros((36, context_size))), axis=1)
            f_forget = np.concatenate((np.zeros((36, 300)), np.random.choice([0, 1], size=(36, context_size), replace=True, p=[1-f, f])), axis=1)
            r_mem[r_encode == 1] = 1
            f_mem[f_encode == 1] = 1
            f_mem[f_forget == 1] = 0
        mem[:36] *= r_mem
        mem[36:] *= f_mem
        fams = echo_intensity(probes, mem, tau=t)
        criterion = np.percentile(fams, 100-c[2])
        r_hits = np.sum(fams[:36] > criterion) / 36
        f_hits = np.sum(fams[36:72] > criterion) / 36
        r_fa = np.sum(fams[72:90] > criterion) / 18
        f_fa = np.sum(fams[90:108] > criterion) / 18
        u_fa = np.sum(fams[108:144] > criterion) / 36
        sim_list.append([r_hits, f_hits, r_fa, f_fa, u_fa])
    sim_means = np.mean(sim_list, axis=0)
    sds.append(np.std(sim_list, axis=0, ddof=1))
    print(c[3], sim_means)
    overall_means.append(sim_means)

            Unbinding Simulation MEANS

condition    R old      F old      R rel      F rel      Unrel

short/short [0.58319444 0.52094444 0.43105556 0.40377778 0.284     ]
short/long  [0.72994444 0.53275    0.28111111 0.19855556 0.10858333]
long/short  [0.69130556 0.52580556 0.26511111 0.19688889 0.10744444]
long/long   [0.79163889 0.55208333 0.24688889 0.14566667 0.07111111]


In [29]:
# print SDS
print('            Unbinding Simulation Standard Deviations\n')
print('condition    R old      F old      R rel      F rel      Unrel\n')
for i in range(0, 4):
    print(cond[i][3], sds[i])

            Unbinding Simulation Standard Deviations

condition    R old      F old      R rel      F rel      Unrel

short/short [0.0816573  0.07965835 0.12429958 0.12523136 0.08503447]
short/long  [0.06303699 0.07144655 0.09799709 0.07188378 0.04600945]
long/short  [0.06529702 0.07197969 0.09217483 0.06964716 0.04240919]
long/long   [0.05431833 0.06573995 0.11165012 0.06864299 0.03899163]


In [31]:
# compute model fits
sim_all = []
for x in overall_means:
    sim_all.extend(x)
print('Context unbinding fits\n')
print('R-squared: ', r_sq(np.array(sim_all), emp_all))
print('Mean Squared Error: ', mse(np.array(sim_all), emp_all))

Context unbinding fits

R-squared:  0.9743964013229056
Mean Squared Error:  0.001944838462277374
